In [1]:
import pandas as pd
import numpy as np

from sklearn import preprocessing

# from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit
# from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.naive_bayes import BernoulliNB
from sklearn import svm

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [2]:
df1 = pd.read_csv('./data/processed_data/all/resample_to_50_per_act/time_features/time_w50.csv')
df2 = pd.read_csv('./data/processed_data/all/resample_to_50_per_act/frequency_features/freq_w50.csv').drop('label',axis=1)
df = pd.concat([df1, df2], axis=1)  

In [3]:
df1.shape
df2.shape
df.shape

(5026, 58)

(5026, 33)

(5026, 91)

In [4]:
df.columns

Index(['label', 'sum_x', 'sum_y', 'sum_z', 'mean_x', 'mean_y', 'mean_z',
       'std_x', 'std_y', 'std_z', 'cv_x', 'cv_y', 'cv_z', 'ptp_x', 'ptp_y',
       'ptp_z', 'p10_x', 'p10_y', 'p10_z', 'p25_x', 'p25_y', 'p25_z', 'p50_x',
       'p50_y', 'p50_z', 'p75_x', 'p75_y', 'p75_z', 'p90_x', ' p90_y', 'p90_z',
       'iq_x', 'iq_y', 'iq_z', 'lagone_autocor_x', 'lagone_autocor_y',
       'lagone_autocor_z', 'sk_x', 'sk_y', 'sk_z', 'kurt_x', 'kurt_y',
       'kurt_z', 'power_x', 'power_y', 'power_z', 'logpower_x', 'logpower_y',
       'logpower_z', 'peak_x', 'peak_y', 'peak_z', 'cross0_x', 'cross0_y',
       'cross0_z', 'cor_xy', 'cor_yz', 'cor_zx', 'dc_x', 'dc_y', 'dc_z',
       'mean_x', 'mean_y', 'mean_z', 'var_x', 'var_y', 'var_z', 'std_x',
       'std_y', 'std_z', 'skew_x', 'skew_y', 'skew_z', 'kurt_x', 'kurt_y',
       'kurt_z', 'shape_mean_x', 'shape_mean_y', 'shape_mean_z', 'shape_var_x',
       'shape_var_y', 'shape_var_z', 'shape_std_x', 'shape_std_y',
       'shape_std_z', 'shape_

In [5]:
df_data = np.array(df.drop('label',axis=1))

In [6]:
df_data.shape

(5026, 90)

In [7]:
df_labels = df['label']

In [8]:
df_labels.shape

(5026,)

In [9]:
train_data, test_data, train_labels, test_labels = train_test_split(df_data, df_labels, test_size=0.1)

### 1. kNN

In [10]:
#knn = KNeighborsClassifier(n_neighbors=5, weights='distance')
knn = KNeighborsClassifier()

In [11]:
# 对数据集进行指定次数的交叉验证并为每次验证效果评测
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
# cv = KFold(n_splits=20)
scores = cross_val_score(knn, df_data, df_labels, cv=cv)

In [12]:
scores

array([ 0.85884692,  0.89463221,  0.91053678,  0.90059642,  0.88866799,
        0.90457256,  0.88270378,  0.88469185,  0.89662028,  0.88071571])

In [13]:
scores.mean()

0.89025844930417486

In [14]:
knn.fit(train_data, train_labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [15]:
knn_labels = knn.predict(test_data)

In [16]:
knn_labels.size

503

In [17]:
test_labels.size

503

In [18]:
#knn_labels[0] == test_labels[0]

In [19]:
print(classification_report(test_labels, knn_labels,digits=5))

             precision    recall  f1-score   support

      bweep    0.87578   0.93377   0.90385       151
      clean    0.98214   0.88710   0.93220        62
      daily    0.85294   0.85294   0.85294       102
       dump    0.92857   0.76471   0.83871        51
        run    1.00000   0.66667   0.80000         3
      sweep    0.91200   0.95798   0.93443       119
       walk    0.86667   0.86667   0.86667        15

avg / total    0.89865   0.89662   0.89592       503



### 2. DesicisionTree

In [20]:
dt = DecisionTreeClassifier(criterion='entropy')

In [21]:
# 对数据集进行指定次数的交叉验证并为每次验证效果评测
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
scores = cross_val_score(dt, df_data, df_labels, cv=cv)

In [22]:
scores

array([ 0.8389662 ,  0.83499006,  0.82703777,  0.82703777,  0.84493042,
        0.81312127,  0.80119284,  0.83101392,  0.81510934,  0.85089463])

In [23]:
scores.mean()

0.82842942345924464

In [24]:
dt.fit(train_data, train_labels)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [25]:
dt_labels = dt.predict(test_data)

In [26]:
print(classification_report(test_labels, dt_labels, digits=5))

             precision    recall  f1-score   support

      bweep    0.86577   0.85430   0.86000       151
      clean    0.81967   0.80645   0.81301        62
      daily    0.86170   0.79412   0.82653       102
       dump    0.77778   0.82353   0.80000        51
        run    0.50000   0.66667   0.57143         3
      sweep    0.88889   0.94118   0.91429       119
       walk    0.86667   0.86667   0.86667        15

avg / total    0.85366   0.85288   0.85266       503



### 3. Naive Bayes

In [27]:
gnb = GaussianNB()
#gnb = BernoulliNB()

In [28]:
# 对数据集进行指定次数的交叉验证并为每次验证效果评测
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
scores = cross_val_score(gnb, df_data, df_labels, cv=cv)

In [29]:
scores

array([ 0.61431412,  0.56063618,  0.61033797,  0.64214712,  0.60437376,
        0.57455268,  0.57852883,  0.56858847,  0.5944334 ,  0.58250497])

In [30]:
scores.mean()

0.59304174950298205

In [31]:
gnb.fit(train_data, train_labels)

GaussianNB(priors=None)

In [32]:
gnb_labels = gnb.predict(test_data)

In [33]:
print(classification_report(test_labels, gnb_labels, digits=5))

             precision    recall  f1-score   support

      bweep    0.71795   0.37086   0.48908       151
      clean    0.56164   0.66129   0.60741        62
      daily    0.69737   0.51961   0.59551       102
       dump    0.65079   0.80392   0.71930        51
        run    0.40000   0.66667   0.50000         3
      sweep    0.64881   0.91597   0.75958       119
       walk    0.35000   0.93333   0.50909        15

avg / total    0.65847   0.62823   0.61325       503



### 4. SVM

In [34]:
clf = svm.SVC(C=5, cache_size=1000)

In [35]:
# 对数据集进行指定次数的交叉验证并为每次验证效果评测
# cv = ShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
# cv = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.1)
scores = cross_val_score(clf, df_data, df_labels, cv=cv)

In [36]:
scores

array([ 0.91053678,  0.91650099,  0.94234592,  0.91252485,  0.92047714,
        0.90656064,  0.94035785,  0.90059642,  0.92246521,  0.91451292])

In [37]:
scores.mean()

0.91868787276341934

In [38]:
clf.fit(train_data, train_labels)

SVC(C=5, cache_size=1000, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [39]:
#这个计算出的实际上是recall的加权平均值
clf.score(test_data, test_labels)

0.93041749502982107

In [40]:
clf_labels =clf.predict(test_data)

In [41]:
#这个计算的是f1-score的加权平均值
f1_score(test_labels, clf_labels, average='weighted')  

0.93005174762562626

In [42]:
print(classification_report(test_labels, clf_labels,digits=5))

             precision    recall  f1-score   support

      bweep    0.94631   0.93377   0.94000       151
      clean    1.00000   0.90323   0.94915        62
      daily    0.88889   0.94118   0.91429       102
       dump    0.89362   0.82353   0.85714        51
        run    1.00000   0.66667   0.80000         3
      sweep    0.92857   0.98319   0.95510       119
       walk    0.93333   0.93333   0.93333        15

avg / total    0.93168   0.93042   0.93005       503

